In [187]:
# Importar las librerias necesarias
import json 
import spacy 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity 


In [ ]:
# Carga del modelo de lenguaje en Ingles de spaCy 
nlp = spacy.load("en_core_web_md")

# Leer las respuestas desde el archivo JSON
with open('respuesta.json') as file:
    data = json.load(file)
    responses = data["responses"]

In [189]:

# Dataset transformado en  embeddings
responses_embeddings = np.array([nlp(response).vector for response in responses])

# Funcion para buscar las  respuestas más similares a la pregunta del usuario
def find_most_similar_response(question):
    question_embedding = nlp(question).vector

    # Calcula la similitud de coseno entre la pregunta y las respuestas
    similarities = cosine_similarity([question_embedding], responses_embeddings)

    # obtiene los indices de las 10 respuestas mas similares
    top_indices = np.argsort(similarities[0])[-10:][::-1]

    # Selecciona la respuesta con mayor similitud entre las 10 respuestas
    selected_response = responses[top_indices[0]]

    return selected_response

print("User Question:", user_question)
print("Selected Response:", selected_response)
# Preguntas de ejemplo
example_questions = [
    "What are the visiting hours?",
    "When can I visit the patients?",
    "How can I schedule a medical appointment?",
    "Is the emergency room open 24 hours?",
    "What is the number to schedule an appointment?",
    "Physical therapy sessions?",
    "Is there free WiFi?",
    "Is there quiet hours at the hospital?",
    
]
# Prueba el chatbot con las preguntas de ejemplo
for question in example_questions:
    selected_response = find_most_similar_response(question)
    print("Question: " ,question)
    print("Selected response: " ,selected_response)
    print("--------------------------------------------------")

User Question: What are the visiting hours?
Selected Response: Patients can have two visitors at a time during visiting hours.
Question:  What are the visiting hours?
Selected response:  Family members can call the patient's room directly for updates.
--------------------------------------------------
Question:  When can I visit the patients?
Selected response:  Family members can call the patient's room directly for updates.
--------------------------------------------------
Question:  How can I schedule a medical appointment?
Selected response:  Family members can call the patient's room directly for updates.
--------------------------------------------------
Question:  Is the emergency room open 24 hours?
Selected response:  The emergency room is open 24 hours a day.
--------------------------------------------------
Question:  What is the number to schedule an appointment?
Selected response:  Family members can call the patient's room directly for updates.
-------------------------